In [3]:
from math import log
import numpy as np

In [4]:
def cal_EAR_from_APR(apr, n):
    """
    Calculate the effective annual rate (EAR) from the annual percentage rate (APR) and the number of compounding periods per year (n).
    """
    return (1 + apr/n) ** n - 1

def cal_APR_from_EAR(ear, n):
    """
    Calculate the annual percentage rate (APR) from the effective annual rate (EAR) and the number of compounding periods per year (n).
    """
    return ((1 + ear) ** (1/n) - 1) * 12


In [5]:
def cal_PV_from_FV(FV, r, n):
    """
    FV: future value
    r: period interest rate
    n: number of periods
    """
    return FV / (1 + r) ** n

# example
FV = 2155000
r = 0.0492 / 12 # 4.92% apr
n = 10 # 10 months
cal_PV_from_FV(FV, r, n)

2068605.1602565811

In [6]:
def cal_period_needed(FV, r, PV):
    """
    FV: future value
    r: period interest rate
    PV: present value
    """
    return log(FV/PV) / log(1+r)

# example
PV = 150725
ear = 0.084
FV = 320000

cal_period_needed(FV, ear, PV)


9.334038987542009

In [7]:
def cal_FV_from_PV(PV, r, n):
    """
    PV: present value
    r: period interest rate
    n: number of periods
    """
    return PV * (1 + r) ** n

# example
PV = 200000
apr = 0.08
n = 10
cal_FV_from_PV(PV, apr, n)

431784.99945455766

In [8]:
def cal_r_from_FV_PV(FV, PV, n):
    """
    FV: future value
    PV: present value
    n: number of periods
    """
    return (FV/PV) ** (1/n) - 1

# example
PV = 10000
n = 3
FV = 12600
cal_r_from_FV_PV(FV, PV, n)

0.08008229825529067

In [9]:
def cal_monthly_payment(PV, r, n):
    """
    PV: present value
    r: period interest rate
    n: number of periods
    """
    return PV * r * (1 + r) ** n / ((1 + r) ** n - 1)

# example
PV = 16000
n = 36
r = 0.054/12
cal_monthly_payment(PV, r, n)

482.413093630601

In [1]:
def cal_FV_with_deposit(term, deposit, r):
    """
    term: number of periods
    deposit: deposit amount each period
    r: period interest rate
    """
    return deposit * ((1 + r) ** term - 1) / r

# example
deposit = 200
terms = 40*12
r = 0.1/12
cal_FV_with_deposit(terms, deposit, r)

1264815.9161838996

Cash Flows to be considered:

Revenue + financing (selling bond or stock) = Expense for Operating + debet service + investment in new assets + Divident

Operating Cash Flow = Revenue - Expense for Operating

In [11]:
def cal_OCF(sale, cost, tax_rate, depreciation):
    """
    sale: sale in the period
    cost: cost in the period
    tax_rate: tax rate
    depreciation: depreciation in the period
    """
    return (sale - cost) * (1 - tax_rate) + depreciation * tax_rate

sale = 2.1 * 1e6
cost = 1.55 * 1e6
tax_rate = 0.3
depreciation = 3 * 1e6 / 6
cal_OCF(sale, cost, tax_rate, depreciation)

535000.0

In [12]:
def cal_IRR(initial_investment, cash_flows, n):
    """
    initial_investment: initial investment
    cash_flows: list of cash flows in each period
    n: number of periods
    """
    irr = 0.1  # Initial guess for IRR
    epsilon = 1e-6  # Convergence tolerance
    max_iterations = 1000  # Maximum number of iterations

    for _ in range(max_iterations):
        npv = -initial_investment
        npv_derivative = 0
        for t in range(1, n + 1):
            npv += cash_flows[t-1] / (1 + irr) ** t
            npv_derivative -= t * cash_flows[t-1] / (1 + irr) ** (t + 1)

        if abs(npv) < epsilon:
            return irr

        irr -= npv / npv_derivative

    raise ValueError("IRR calculation did not converge")

initial_investment = 10 * 1e6
n = 10
cash_flows = [1.25 * 1e6] * n  # Create a list with n same cash flows
cal_IRR(initial_investment, cash_flows, n)

0.04277497803511139

In [ ]:
def cal_investment_NPV_IRR(investment, life, salvage, sales, cost, r, tax, depreciation):
    """
    investment: initial investment
    life: life of the project
    salvage: salvage value
    sales: sales revenue
    cost: cost
    r: required rate of return
    tax: tax rate
    depreciation: depreciation amount in each year
    """
    ocf = cal_OCF(sales, cost, tax, depreciation)
    pv_ocf = ocf * (1 - (1 + r) ** -life) / r
    book_value = investment - depreciation * life
    ATSV = salvage - (tax * (salvage - book_value))
    pv_ATSV = ATSV / (1 + r) ** life
    NPV = -investment + pv_ocf + pv_ATSV
    list_ocf = [ocf] * life 
    IRR = cal_IRR(investment, list_ocf, life)
    return NPV, IRR

investment = 12000000
n = 12
depreciation = investment / n
salvage = 1200000
sale = 4150000
cost = 1300000
r = 0.13
tax = 0.28
cal_investment_NPV_IRR(investment, n, salvage, sale, cost, r, tax, depreciation)

1000000.0

Now start the most difficulty part of bonds, the annuity: a stream of constant cash flows (C) with a finite life (t).

While in constrast, perpetuity is a constant cash flow stream with infinite life.

So that basic formula: $PV_{ann} = PV_{reg perp} - PV_{delay perp} = \frac{C}{r}(1-\frac{1}{(1+r)^t})$

In [14]:
def plan_retirement_period_savings(initial_savings, r, n, goal):
    """
    initial_savings: initial savings
    r: effective interest rate for each period
    n: number of periods
    goal: goal amount
    """
    FV_initial = initial_savings * (1 + r) ** n
    FV_needed = goal - FV_initial
    period_savings = FV_needed * r / ((1 + r) ** n - 1)
    return period_savings

initial_savings = 180000
EAR = 0.0925
n = 30 * 12
goal = 4.3 * 1e6

plan_retirement_period_savings(initial_savings, cal_APR_from_EAR(EAR, 12) / 12, n, goal)

975.6180209447112

In [15]:
def cal_r_from_period_payment(period_payment, n, PV, down_payment=0):
    """
    Calculates the periodic effective interest rate using the Bisection Method.

    This function does not use any external packages like numpy. It manually
    implements a numerical solver to find the interest rate.

    Args:
        period_payment (float): The amount of the payment made each period.
        n (int): The total number of payment periods.
        PV (float): The present value or the initial principal of the loan/asset.
        down_payment (float, optional): Any down payment made at the beginning. 
                                        Defaults to 0.

    Returns:
        float: The periodic interest rate, or None if a solution cannot be found.
    """
    loan_amount = PV - down_payment

    # --- Basic Sanity Checks ---
    if n <= 0:
        return None # Number of periods must be positive
    # If total payments are less than the loan, it implies a negative interest rate,
    # which this simple solver isn't designed to handle.
    if period_payment * n <= loan_amount:
        return None

    # --- Objective Function ---
    # We need to find the root `r` for the Present Value of an Annuity formula:
    # f(r) = period_payment * (1 - (1 + r)**-n) / r - loan_amount = 0
    # This is the function whose root (where f(r) = 0) we are trying to find.
    def f(r):
        if r == 0: # Avoid division by zero; the limit of the formula as r->0 is n * pmt
            return (period_payment * n) - loan_amount
        else:
            return period_payment * (1 - (1 + r)**-n) / r - loan_amount

    # --- Bisection Method Implementation ---
    low = 0.0
    high = 1.0  # An upper bound of 100% periodic rate is a safe starting point
    tolerance = 1e-7 # The desired precision for the result
    max_iterations = 100 # A safeguard against an infinite loop

    # The bisection method requires the function to have opposite signs at the
    # low and high ends of the bracket. For a valid loan (r>0), f(low) will be
    # positive and f(high) will be negative.
    if f(low) * f(high) > 0:
        # If signs are the same, a simple root may not exist in this bracket.
        # This can happen for unusual loan terms.
        return None

    for _ in range(max_iterations):
        mid_rate = (low + high) / 2
        f_mid = f(mid_rate)

        # If the value of the function at the midpoint is very close to zero,
        # we have found our root.
        if abs(f_mid) < tolerance:
            return mid_rate

        # If the sign of f(mid_rate) is the same as f(low), it means the root
        # lies in the upper half of the interval. So, we move the lower bound up.
        elif f(low) * f_mid > 0:
            low = mid_rate
        # Otherwise, the root is in the lower half. We move the upper bound down.
        else:
            high = mid_rate
            
    # If the loop completes without converging, return None.
    return None

PV = 425000
n = 360
monthly_payment = 2346.87

cal_r_from_period_payment(monthly_payment, n, PV) * 12

0.052500162056048794

(1999282.7305999605, 0.16239606942510681)